In [12]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
from joblib import Parallel, delayed
from tqdm import tqdm

In [13]:
df = pd.read_parquet('../../../metrics.parquet')

In [14]:
df, _ = train_test_split(df, train_size=200000, stratify=df['time_to_stop_activity'], random_state=11)

In [15]:
df = df.drop(
    columns = [
        "user", 
        'project', 
        'current_month', 
        'turnover_num',
        'turnover',
        'betweenness_centrality_12_intercept',
        'betweenness_centrality_12_slope', 
        'betweenness_centrality_12_std_dev',
        'betweenness_centrality_3_intercept', 
        'betweenness_centrality_3_slope',
        'betweenness_centrality_3_std_dev',
        'betweenness_centrality_6_intercept', 
        'betweenness_centrality_6_slope',
        'betweenness_centrality_6_std_dev',
        'betweenness_centrality_9_intercept', 
        'betweenness_centrality_9_slope',
        'betweenness_centrality_9_std_dev'
    ], axis = 1
)

In [16]:
df = df.astype(np.float64)
df = df.replace([np.inf], np.nan)

for column in df.columns:
    max_value = df[column].max(skipna=True) 
    df[column] = df[column].fillna(max_value)

df = df.replace([-np.inf], np.nan)

for column in df.columns:
    max_value = df[column].min(skipna=True) 
    df[column] = df[column].fillna(max_value)

In [17]:
for column in df.columns:
    max_value = df[column].max(skipna=True) 
    df[column] = df[column].fillna(max_value)

In [18]:
def train_evaluate_gb(n_estimators, learning_rate, max_depth, X_train, Y_train, X_test, Y_test):
    try:
        gb = GradientBoostingRegressor(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            random_state=11
        )
        
        gb.fit(X_train, Y_train)
        y_pred = gb.predict(X_test)

        mse = mean_squared_error(Y_test, y_pred)
        mae = mean_absolute_error(Y_test, y_pred)
        r2 = r2_score(Y_test, y_pred)

        print(
            f"GB - Estimators: {n_estimators}, Learning Rate: {learning_rate}, Max Depth: {max_depth} "
            f"Finalized - MSE: {mse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}"
        )
        
        return {
            'n_estimators': n_estimators,
            'learning_rate': learning_rate,
            'max_depth': max_depth,
            'mean_squared_error': mse,
            'mean_absolute_error': mae,
            'r2_score': r2
        }
    
    except Exception as e:
        print(f"GB - Estimators: {n_estimators}, Learning Rate: {learning_rate}, Max Depth: {max_depth} Error: {str(e)}")

        return {
            'n_estimators': n_estimators,
            'learning_rate': learning_rate,
            'max_depth': max_depth,
            'mean_squared_error': None,
            'mean_absolute_error': None,
            'r2_score': None
        }

In [19]:
param_combinations = [
    (n_estimators, learning_rate, max_depth)
    for n_estimators in [50, 100, 200]
    for learning_rate in [0.01, 0.1, 0.2]
    for max_depth in [3, 5, 10]
]

In [20]:
x = df.drop(columns = ['time_to_stop_activity'], axis = 1).values
y = df['time_to_stop_activity'].values

In [21]:
X_train, X_test, Y_train, Y_test = train_test_split(
    x, y, 
    test_size = 0.3, 
    random_state=11
)

In [22]:
benchmark = Parallel(n_jobs=-1)(
    delayed(train_evaluate_gb)(n_estimators, lr, depth, X_train, Y_train, X_test, Y_test)
    for n_estimators, lr, depth in param_combinations
)

GB - Estimators: 50, Learning Rate: 0.01, Max Depth: 3 Finalized - MSE: 10.7467, MAE: 2.7165, R²: 0.3805
GB - Estimators: 50, Learning Rate: 0.1, Max Depth: 3 Finalized - MSE: 6.0443, MAE: 1.2390, R²: 0.6515
GB - Estimators: 50, Learning Rate: 0.2, Max Depth: 3 Finalized - MSE: 5.9092, MAE: 1.1853, R²: 0.6593
GB - Estimators: 50, Learning Rate: 0.01, Max Depth: 5 Finalized - MSE: 10.3271, MAE: 2.6655, R²: 0.4046
GB - Estimators: 50, Learning Rate: 0.1, Max Depth: 5 Finalized - MSE: 5.7759, MAE: 1.1558, R²: 0.6670
GB - Estimators: 50, Learning Rate: 0.2, Max Depth: 5 Finalized - MSE: 5.7194, MAE: 1.1361, R²: 0.6703
GB - Estimators: 100, Learning Rate: 0.01, Max Depth: 3 Finalized - MSE: 8.1900, MAE: 2.1535, R²: 0.5278
GB - Estimators: 100, Learning Rate: 0.1, Max Depth: 3 Finalized - MSE: 5.8993, MAE: 1.1872, R²: 0.6599
GB - Estimators: 100, Learning Rate: 0.2, Max Depth: 3 Finalized - MSE: 5.8163, MAE: 1.1643, R²: 0.6647
GB - Estimators: 50, Learning Rate: 0.01, Max Depth: 10 Finalized

In [23]:
benchmark_df = pd.DataFrame(benchmark)

In [24]:
benchmark_df.to_excel('GradientBoostRegressorBenchmark.xlsx')